asdf

In [2]:
# Liste der Forscher:innen, für die die Wikidata-Einträge überprüft werden sollen
search_terms = ["Förstner, Konrad", "Becker, Anke", "Bork, Peer", "Clavel, Thomas", "Goesmann, Alexander"]

## Q-IDs ermitteln
Dieses Skript durchsucht Wikidata nach bestimmten Namen, um ihre entsprechenden QIDs (Wikidata-IDs) zu finden und die zugehörigen Informationen abzurufen. Bei mehreren gefundenen Einträgen für einen Namen wird der Benutzer zur Auswahl der richtigen QID aufgefordert. Die Informationen über Personen (Menschen) werden dann in einem Dictionary gespeichert und ausgegeben.


In [11]:
import requests

# Dictionary zur Speicherung von Informationen zu Menschen
human_info_dict = {}

# Funktion zur Auswahl einer QID bei mehreren Ergebnissen für denselben Suchbegriff
def choose_qid(qid_options, name):
    if len(qid_options) == 1:
        chosen_qid = qid_options[0]
        print(f"Nur ein Eintrag gefunden für '{name}': QID: {chosen_qid}")
        return chosen_qid
    else:
        print(f"Mehrere Einträge gefunden für '{name}'. Bitte wählen Sie eine QID aus:")
        valid_entries = []
        for idx, qid in enumerate(qid_options):
            entity_url = f"https://www.wikidata.org/wiki/Special:EntityData/{qid}.json"
            try:
                response = requests.get(entity_url)
                entity_data = response.json()
                if "entities" in entity_data and qid in entity_data["entities"]:
                    entity_info = entity_data["entities"][qid]
                    label = entity_info.get('labels', {}).get('en', {}).get('value', 'N/A')
                    description = entity_info.get('descriptions', {}).get('en', {}).get('value', 'N/A')
                    instance_of_claims = entity_info["claims"].get("P31", [])
                    is_human = any(claim["mainsnak"]["datavalue"]["value"]["id"] == "Q5" for claim in instance_of_claims)
                    if is_human:
                        valid_entries.append({
                            'qid': qid,
                            'label': label,
                            'description': description
                        })
                        print(f"{idx + 1}: QID: {qid}, Label: {label}, Beschreibung: {description}")
            except requests.exceptions.RequestException as e:
                print(f"Fehler bei der Abfrage für {qid}: {e}")

        while True:
            choice = input("Geben Sie die Nummer der gewünschten QID ein: ")
            try:
                index = int(choice) - 1
                if 0 <= index < len(valid_entries):
                    return valid_entries[index]['qid']
                else:
                    print("Ungültige Eingabe. Bitte wählen Sie eine der angezeigten Optionen.")
            except ValueError:
                print("Ungültige Eingabe. Bitte geben Sie eine Zahl ein.")

# Durchlauf der Suchbegriffe
for search_term in search_terms:
    api_url = f"https://www.wikidata.org/w/api.php?action=query&format=json&list=search&srsearch={search_term}"

    try:
        # HTTP-Anfrage, um die Suchergebnisse von Wikidata abzurufen
        response = requests.get(api_url)
        data = response.json()

        # Überprüfen der Ergebnisse und Extrahieren der QIDs
        if "query" in data and "search" in data["query"]:
            qids_options = []
            for result in data["query"]["search"]:

                q_id = result["title"]
                qids_options.append(q_id)

            # Auswahl einer QID, falls mehrere Optionen vorhanden sind
            if len(qids_options) > 0:
                chosen_qid = choose_qid(qids_options, search_term)

                # API-Anfrage, um die JSON-Daten des Wikidata-Eintrags abzurufen
                entity_url = f"https://www.wikidata.org/wiki/Special:EntityData/{chosen_qid}.json"
                entity_response = requests.get(entity_url)
                entity_data = entity_response.json()

                # Überprüfen, ob der Eintrag eine Instanz der Klasse "Mensch" (Q5) ist
                if "entities" in entity_data and chosen_qid in entity_data["entities"]:
                    entity_info = entity_data["entities"][chosen_qid]

                    # Überprüfen, ob der Eintrag einen Menschen repräsentiert
                    instance_of_claims = entity_info["claims"].get("P31", [])
                    is_human = any(claim["mainsnak"]["datavalue"]["value"]["id"] == "Q5" for claim in instance_of_claims)

                    if is_human:
                        label = entity_info.get('labels', {}).get('en', {}).get('value', 'N/A')
                        description = entity_info.get('descriptions', {}).get('en', {}).get('value', 'N/A')

                        # Speichern der Informationen im Dictionary
                        human_info_dict[label] = {
                            'description': description,
                            'qid': chosen_qid
                        }
                    else:
                        print(f"Der Wikidata-Eintrag {chosen_qid} repräsentiert keine Person (Mensch).")
                else:
                    print(f"Der Wikidata-Eintrag {chosen_qid} wurde nicht gefunden oder enthält keine Daten.")

            else:
                print(f"Keine Ergebnisse gefunden für den Suchbegriff '{search_term}'.")

    except requests.exceptions.RequestException as e:
        print(f"Fehler bei der HTTP-Anfrage für {search_term}: {e}")
    except KeyError as e:
        print(f"Unerwarteter JSON-Formatfehler für {search_term}: {e}")
    except Exception as e:
        print(f"Allgemeiner Fehler für {search_term}: {e}")

# Ausgabe des Dictionarys mit Informationen zu Menschen
print("Dictionary mit Informationen zu Menschen:")
for name, info in human_info_dict.items():
    print(f"Name: {name}, Beschreibung: {info['description']}, QID: {info['qid']}")

Mehrere Einträge gefunden für 'Förstner, Konrad'. Bitte wählen Sie eine QID aus:
1: QID: Q18744528, Label: Konrad Förstner, Beschreibung: bioinformatician
Mehrere Einträge gefunden für 'Becker, Anke'. Bitte wählen Sie eine QID aus:
1: QID: Q21253882, Label: Anke Becker, Beschreibung: German university teacher
2: QID: Q124022504, Label: Anke Becker, Beschreibung: German visual artist
3: QID: Q124413921, Label: Anke Becker, Beschreibung: faculty at Harvard Business School
Mehrere Einträge gefunden für 'Bork, Peer'. Bitte wählen Sie eine QID aus:
1: QID: Q7160367, Label: Peer Bork, Beschreibung: German biologist and bioinformatician
Mehrere Einträge gefunden für 'Clavel, Thomas'. Bitte wählen Sie eine QID aus:
1: QID: Q40442760, Label: Thomas Clavel, Beschreibung: researcher
2: QID: Q96748697, Label: Thomas Clavel, Beschreibung: French writer
Mehrere Einträge gefunden für 'Goesmann, Alexander'. Bitte wählen Sie eine QID aus:
1: QID: Q52422599, Label: Alexander Goesmann, Beschreibung: rese

In [12]:
print(human_info_dict)

{'Konrad Förstner': {'description': 'bioinformatician', 'qid': 'Q18744528'}, 'Anke Becker': {'description': 'German university teacher', 'qid': 'Q21253882'}, 'Peer Bork': {'description': 'German biologist and bioinformatician', 'qid': 'Q7160367'}, 'Thomas Clavel': {'description': 'researcher', 'qid': 'Q40442760'}, 'Alexander Goesmann': {'description': 'researcher', 'qid': 'Q52422599'}}


## Abrufen und Speichern von GEPRIS-Daten für Wissenschaftler
In diesem Notebook wird eine Liste von Wissenschaftlern durch die GEPRIS-Datenbank durchsucht, um ihre Projektinformationen abzurufen. Der Code verwendet requests und BeautifulSoup, um die HTML-Inhalte der Suchergebnisse zu parsen, und speichert die gefundenen Projektdaten in JSON-Dateien. Die extrahierten Informationen werden in einem speziell erstellten Ordner abgelegt.

In [3]:
import requests
from bs4 import BeautifulSoup
import json
import re  # Regex-Modul zum Bereinigen des Dateinamens
import os  # Modul zum Arbeiten mit dem Dateisystem

def get_gepris_id_for_person(name):
    search_url = "https://gepris.dfg.de/gepris/OCTOPUS"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }
    params = {
        "context": "person",
        "task": "doSearchSimple",
        "keywords_criterion": name,
        "findButton": "historyCall",
        "hitsPerPage": 10,
        "index": 0,
        "nurProjekteMitAB": False
    }

    try:
        response = requests.get(search_url, params=params, headers=headers)
        response.raise_for_status()  # Raise an exception for bad responses
        soup = BeautifulSoup(response.content, 'html.parser')

        # Find the first search result link containing the name
        search_results = soup.find_all("a", href=True)
        for result in search_results:
            if name.lower() in result.text.lower():  # Case-insensitive comparison
                return result['href']

        print(f"No GEPRIS ID found for {name}")
        return None
    
    except requests.exceptions.RequestException as e:
        print(f"Error accessing GEPRIS for {name}: {e}")
        return None

# Liste für die Ergebnisse initialisieren
gepris_id_links = []

# Durchlaufe jeden Suchbegriff und rufe die GEPRIS-ID ab
for person_name in search_terms:
    gepris_id_link = get_gepris_id_for_person(person_name)
    if gepris_id_link:
        # Vervollständige die URL mit dem Basis-Link, falls nötig
        if not gepris_id_link.startswith("https://"):
            gepris_id_link = f"https://gepris.dfg.de{gepris_id_link}"
        gepris_id_links.append(gepris_id_link)
        print(f"GEPRIS ID Link für {person_name}: {gepris_id_link}")
    else:
        print(f"GEPRIS ID Link für {person_name} konnte nicht gefunden werden.")

# Überprüfe, ob der Ordner 'jsonGepris' existiert, und erstelle ihn bei Bedarf
output_folder = "GeprisJsonAndCsv"
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Durchlaufe die Liste der GEPRIS-ID-Links
for url in gepris_id_links:
    try:
        # Initialisiere ein leeres Dictionary für die Person und ihre Projekte
        person_data = {}

        # Führe eine HTTP-Anfrage durch, um den HTML-Inhalt der Seite zu erhalten
        response = requests.get(url)

        # Überprüfe, ob die Anfrage erfolgreich war (Status-Code 200)
        if response.status_code == 200:
            # Parsen des HTML-Inhalts mit Beautiful Soup
            soup = BeautifulSoup(response.content, 'html.parser')

            # Extrahiere den Namen der Person (aus dem Titel der Seite)
            person_name = soup.title.text.strip().replace("DFG - GEPRIS - ", "")

            # Bereinige den Personennamen für den Dateinamen (entferne Sonderzeichen und Leerzeichen)
            clean_person_name = re.sub(r'[^\w\s-]', '', person_name).strip()
            clean_person_name = re.sub(r'[-\s]+', '-', clean_person_name)

            # Füge den bereinigten Namen zur Personendaten hinzu
            person_data["Person Name"] = person_name

            # Finde alle Links auf der Seite
            links = soup.find_all('a', href=True)

            # Filtere nur die Links, die auf Projekte verweisen
            project_links = [link for link in links if '/gepris/projekt/' in link['href']]

            # Extrahiere den Text (Namen) und den Link aus den gefundenen Projekt-Links
            projects_data = []
            for project_link in project_links:
                project_name = project_link.get_text(strip=True)
                project_url = project_link['href']
                projects_data.append({
                    "Projekt Name": project_name,
                    "Projekt Link": project_url
                })

            # Füge die Projekte zum Dictionary der Person hinzu
            person_data["Projekte"] = projects_data

            # Erstelle den Dateinamen mit dem bereinigten Personennamen und füge 'Wikidata' hinzu
            output_file = os.path.join(output_folder, f"{clean_person_name}Gepris.json")

            # Speichere die Daten in einer JSON-Datei mit dem entsprechenden Dateinamen
            with open(output_file, 'w', encoding='utf-8') as json_file:
                json.dump(person_data, json_file, ensure_ascii=False, indent=2)

            print(f"Daten für Person '{person_name}' erfolgreich in '{output_file}' gespeichert.")

        else:
            print(f"Fehler bei der Anfrage für {url}: {response.status_code}")

    except requests.exceptions.RequestException as e:
        print(f"Fehler bei der HTTP-Anfrage für {url}: {e}")
    except Exception as ex:
        print(f"Allgemeiner Fehler für {url}: {ex}")


GEPRIS ID Link für Förstner, Konrad: https://gepris.dfg.de/gepris/person/262159783
GEPRIS ID Link für Becker, Anke: https://gepris.dfg.de/gepris/person/1592887
GEPRIS ID Link für Bork, Peer: https://gepris.dfg.de/gepris/person/1011382
GEPRIS ID Link für Clavel, Thomas: https://gepris.dfg.de/gepris/person/184220620
GEPRIS ID Link für Goesmann, Alexander: https://gepris.dfg.de/gepris/person/188428736
Daten für Person 'Professor Dr. Konrad  Förstner' erfolgreich in 'GeprisJsonAndCsv\Professor-Dr-Konrad-FörstnerGepris.json' gespeichert.
Daten für Person 'Professorin Dr. Anke  Becker' erfolgreich in 'GeprisJsonAndCsv\Professorin-Dr-Anke-BeckerGepris.json' gespeichert.
Daten für Person 'Professor Dr. Peer  Bork' erfolgreich in 'GeprisJsonAndCsv\Professor-Dr-Peer-BorkGepris.json' gespeichert.
Daten für Person 'Professor Dr. Thomas  Clavel' erfolgreich in 'GeprisJsonAndCsv\Professor-Dr-Thomas-ClavelGepris.json' gespeichert.
Daten für Person 'Professor Dr. Alexander  Goesmann' erfolgreich in 'G

Alle Informationen in einer Liste zusammengefasst

In [4]:
# Initialisiere eine leere Liste für alle Personen und ihre Projekte
all_persons_data = []

# Durchlaufe die Liste aller Personen
for url in gepris_id_links:
    # Initialisiere ein leeres Dictionary für die Person und ihre Projekte
    person_data = {}

    # Führe eine HTTP-Anfrage durch, um den HTML-Inhalt der Seite zu erhalten
    response = requests.get(url)

    # Überprüfe, ob die Anfrage erfolgreich war (Status-Code 200)
    if response.status_code == 200:
        # Parsen des HTML-Inhalts mit Beautiful Soup
        soup = BeautifulSoup(response.content, 'html.parser')

        # Extrahiere den Namen der Person (aus dem Titel der Seite)
        person_name = soup.title.text.strip().replace("DFG - GEPRIS - ", "")
        person_data["Person Name"] = person_name

        # Finde alle Links auf der Seite
        links = soup.find_all('a', href=True)

        # Filtere nur die Links, die auf Projekte verweisen
        project_links = [link for link in links if '/gepris/projekt/' in link['href']]

        # Extrahiere den Text (Namen) und den Link aus den gefundenen Projekt-Links
        projects_data = []
        for project_link in project_links:
            project_name = project_link.get_text(strip=True)
            project_url = project_link['href']
            projects_data.append({
                "Projekt Name": project_name,
                "Projekt Link": project_url
            })

        # Füge die Projekte zum Dictionary der Person hinzu
        person_data["Projekte"] = projects_data

        # Füge die Person zum Gesamtdictionary hinzu
        all_persons_data.append(person_data)

    else:
        print(f"Fehler bei der Anfrage für {url}: {response.status_code}")

# Überprüfe, ob der Ordner 'jsonGepris' existiert, und erstelle ihn bei Bedarf
output_folder = "GeprisJsonAndCsv"
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Speichere alle Daten in einer JSON-Datei im Unterordner 'jsonGepris'
output_file = os.path.join(output_folder, 'output_all_persons_names_projects_links.json')
with open(output_file, 'w', encoding='utf-8') as json_file:
    json.dump(all_persons_data, json_file, ensure_ascii=False, indent=2)

# Drucke eine Bestätigungsmeldung
print(f"Alle Daten erfolgreich in '{output_file}' gespeichert.")


Alle Daten erfolgreich in 'GeprisJsonAndCsv\output_all_persons_names_projects_links.json' gespeichert.


## Prüfen ob Projekte bereits auf wikidata angelegt sind

Alle Gepris Projekte der zu überprüfenden Forscher in einer Liste speichern

In [5]:
import json
import os
import requests

# Pfad zur JSON-Datei (relativ zum aktuellen Verzeichnis)
json_file_path = "GeprisJsonAndCsv/output_all_persons_names_projects_links.json"

# Menge zur Speicherung der Projekt-IDs
project_ids_list = []

# Vollständigen Pfad zur JSON-Datei erstellen
full_json_file_path = os.path.join(os.getcwd(), json_file_path)

# JSON-Datei einlesen und Daten laden
with open(full_json_file_path, 'r', encoding='utf-8') as json_file:
    data = json.load(json_file)

# Durchlaufe jeden Eintrag in der Liste
for entry in data:
    # Extrahiere die Projekte aus dem Eintrag
    projects = entry.get('Projekte', [])
    
    # Durchlaufe die Projekte und extrahiere die Projekt-IDs
    for project in projects:
        project_link = project.get('Projekt Link', '')
        
        # Extrahiere die Projekt-ID aus dem Projekt-Link
        if project_link:
            project_id = project_link.split('/')[-1]
            # Füge die Projekt-ID zur Liste hinzu (Vermeidung von Duplikaten durch Set)
            project_ids_list.append(project_id)

# Ausgabe der gesammelten Projekt-IDs
print("Liste der Projekt-IDs:")
print(project_ids_list)

Liste der Projekt-IDs:
['433110396', '492813820', '509313233', '460129525', '521476232', '520751609', '5095944', '5310710', '71821268', '40014398', '198305071', '218318381', '314772579', '431352836', '230488449', '505997786', '5365365', '224619622', '326552732', '460129525', '18572122', '5131248', '290363600', '24122740', '441914366', '460129525', '5413368', '453182863', '453229399', '513892404', '242504939', '316102599', '418004173', '445552570', '516780480', '418598556', '418603037', '424650015', '424795268', '460129525', '447603908', '456668568', '458957343', '319835486', '325443116', '221270173', '183605059', '284237345', '442032008', '460129525', '255821879', '507302435', '270041755', '491261247']


Abgleich Liste und wikidata

In [6]:
found_entries = []

for project in project_ids_list:
    api_url = f"https://www.wikidata.org/w/api.php?action=query&format=json&list=search&srsearch=P4870+{project}"

    response = requests.get(api_url)
    data = response.json()

    if "query" in data and "search" in data["query"]:
        for result in data["query"]["search"]:
            q_id = result["title"]

            # Weitere Abfrage, um das Label des Wikidata-Eintrags abzurufen
            entity_url = f"https://www.wikidata.org/wiki/Special:EntityData/{q_id}.json"
            entity_response = requests.get(entity_url)
            entity_data = entity_response.json()

            if "entities" in entity_data and q_id in entity_data["entities"]:
                entity_info = entity_data["entities"][q_id]
                label = entity_info.get('labels', {}).get('en', {}).get('value', '')  # Das Label des Eintrags

                # Füge das gefundene Ergebnis der Liste hinzu
                found_entries.append({
                    "project_id": project,
                    "q_id": q_id,
                    "label": label
                })

# Ausgabe der gefundenen Einträge
print("Gefundene Einträge:")
for entry in found_entries:
    print(f"Projekt-ID: {entry['project_id']}, Q-ID: {entry['q_id']}, Label: {entry['label']}")

Gefundene Einträge:
Projekt-ID: 460129525, Q-ID: Q99534506, Label: NFDI4Microbiota
Projekt-ID: 5095944, Q-ID: Q116062070, Label: 
Projekt-ID: 71821268, Q-ID: Q116128332, Label: 
Projekt-ID: 40014398, Q-ID: Q116121999, Label: 
Projekt-ID: 5365365, Q-ID: Q116092079, Label: 
Projekt-ID: 460129525, Q-ID: Q99534506, Label: NFDI4Microbiota
Projekt-ID: 18572122, Q-ID: Q116114558, Label: 
Projekt-ID: 5131248, Q-ID: Q116063769, Label: 
Projekt-ID: 441914366, Q-ID: Q98380337, Label: GHGA
Projekt-ID: 460129525, Q-ID: Q99534506, Label: NFDI4Microbiota
Projekt-ID: 460129525, Q-ID: Q99534506, Label: NFDI4Microbiota
Projekt-ID: 442032008, Q-ID: Q98380341, Label: NFDI4Biodiversity
Projekt-ID: 460129525, Q-ID: Q99534506, Label: NFDI4Microbiota


Alle Projekte ohne Eintrag in einer Liste speichern

In [7]:
# Extrahiere die Projekt-IDs aus den gefundenen Einträgen
found_project_ids = {entry['project_id'] for entry in found_entries}

# Erstelle eine neue Liste, die nur die Projekt-IDs enthält, die nicht in den gefundenen Einträgen sind
non_existing_entries = [project_id for project_id in project_ids_list if project_id not in found_project_ids]

# Ausgabe der gefilterten Liste
print("Gefilterte Projekt-IDs:")
print(non_existing_entries)

Gefilterte Projekt-IDs:
['433110396', '492813820', '509313233', '521476232', '520751609', '5310710', '198305071', '218318381', '314772579', '431352836', '230488449', '505997786', '224619622', '326552732', '290363600', '24122740', '5413368', '453182863', '453229399', '513892404', '242504939', '316102599', '418004173', '445552570', '516780480', '418598556', '418603037', '424650015', '424795268', '447603908', '456668568', '458957343', '319835486', '325443116', '221270173', '183605059', '284237345', '255821879', '507302435', '270041755', '491261247']


## json zum Upload vorbereiten

json mit gepris-Projekten reinigen

In [38]:
import json
import re

input_file = 'GeprisJsonAndCsv/output_all_persons_names_projects_links.json'
output_file = 'GeprisJsonAndCsv/output_all_persons_names_projects_cleaned.json'

# Funktion zum Entfernen der Titel, Punkte und doppelten Leerzeichen
def clean_person_name(name):
    # Entferne Titel
    name = re.sub(r'\b(Professor|Professorin|Dr\.?)\b', '', name, flags=re.IGNORECASE)
    # Entferne überflüssige Punkte
    name = re.sub(r'\.', '', name)
    # Entferne doppelte Leerzeichen und überflüssige Leerzeichen am Anfang und Ende
    name = re.sub(r'\s+', ' ', name).strip()
    return name

# Funktion zum Entfernen des spezifischen Parts aus dem Projekt Link
def clean_project_link(link):
    return re.sub(r'^/gepris/projekt/', '', link)

# Lade die JSON-Daten aus einer Datei
def load_json(filename):
    try:
        with open(filename, 'r', encoding='utf-8') as file:
            return json.load(file)
    except FileNotFoundError:
        print(f"Die Datei {filename} wurde nicht gefunden.")
        raise
    except json.JSONDecodeError:
        print(f"Fehler beim Dekodieren der JSON-Datei {filename}.")
        raise

# Speichere die bearbeiteten JSON-Daten in eine Datei
def save_json(data, filename):
    try:
        with open(filename, 'w', encoding='utf-8') as file:
            json.dump(data, file, ensure_ascii=False, indent=2)
    except IOError:
        print(f"Fehler beim Schreiben der Datei {filename}.")
        raise

# Hauptprogramm
def main(input_file, output_file):
    # JSON-Daten laden
    data = load_json(input_file)
    
    # Namen bearbeiten und Projekt Links reinigen
    for person in data:
        person['Person Name'] = clean_person_name(person['Person Name'])
        for projekt in person.get('Projekte', []):
            projekt['Projekt Link'] = clean_project_link(projekt['Projekt Link'])
    
    # Bearbeitete JSON-Daten speichern
    save_json(data, output_file)
    print(f'Daten wurden erfolgreich bearbeitet und in "{output_file}" gespeichert.')

if __name__ == "__main__":
    main(input_file, output_file)


Daten wurden erfolgreich bearbeitet und in "GeprisJsonAndCsv/output_all_persons_names_projects_cleaned.json" gespeichert.


Struktur der json anpassen

In [42]:
import json

input_file = 'GeprisJsonAndCsv/output_all_persons_names_projects_cleaned.json'
output_file = 'GeprisJsonAndCsv/output_restructured_projects.json'

# Lade die JSON-Daten aus einer Datei
def load_json(filename):
    with open(filename, 'r', encoding='utf-8') as file:
        return json.load(file)

# Speichere die bearbeiteten JSON-Daten in eine Datei
def save_json(data, filename):
    with open(filename, 'w', encoding='utf-8') as file:
        json.dump(data, file, ensure_ascii=False, indent=2)

# Hauptprogramm zur Umstrukturierung der JSON-Daten
def restructure_json(data):
    # Schritt 1: Erstelle ein Mapping von Projekt-IDs zu Namen und Teilnehmern
    project_participants = {}
    
    for person in data:
        participant_id = person['Person Name']
        for projekt in person.get('Projekte', []):
            project_name = projekt['Projekt Name']
            project_link = projekt['Projekt Link']
            
            if project_link not in project_participants:
                project_participants[project_link] = {
                    "name": project_name,
                    "gepris_id": project_link,
                    "participants": []
                }
            if participant_id not in project_participants[project_link]["participants"]:
                project_participants[project_link]["participants"].append(participant_id)
    
    # Schritt 2: Konvertiere das Mapping in eine Liste von Projekten
    projects = list(project_participants.values())
    
    return projects

# Hauptfunktion
def main(input_file, output_file):
    data = load_json(input_file)
    structured_data = restructure_json(data)
    save_json(structured_data, output_file)
    print(f'Daten wurden erfolgreich bearbeitet und in "{output_file}" gespeichert.')

if __name__ == "__main__":
    main(input_file, output_file)


Daten wurden erfolgreich bearbeitet und in "GeprisJsonAndCsv/output_restructured_projects.json" gespeichert.


json mit gepris_infos mit den Q-IDs der Forschenden verknüpfen

In [44]:
import json

input_file = 'GeprisJsonAndCsv/output_restructured_projects.json'
output_file = 'GeprisJsonAndCsv/output_restructured_projects_with_ids.json'

# Dictionary mit weiteren Informationen zu den Forschern
human_info_dict = {
    'Konrad Förstner': {'description': 'bioinformatician', 'qid': 'Q18744528'},
    'Anke Becker': {'description': 'German university teacher', 'qid': 'Q21253882'},
    'Peer Bork': {'description': 'German biologist and bioinformatician', 'qid': 'Q7160367'},
    'Thomas Clavel': {'description': 'researcher', 'qid': 'Q40442760'},
    'Alexander Goesmann': {'description': 'researcher', 'qid': 'Q52422599'}
}

# Lade die JSON-Daten aus einer Datei
def load_json(filename):
    with open(filename, 'r', encoding='utf-8') as file:
        return json.load(file)

# Speichere die bearbeiteten JSON-Daten in eine Datei
def save_json(data, filename):
    with open(filename, 'w', encoding='utf-8') as file:
        json.dump(data, file, ensure_ascii=False, indent=2)

# Funktion zur Umstrukturierung der JSON-Daten
def restructure_json(data):
    projects_dict = {}

    for project in data:
        project_name = project['name']
        project_id = project['gepris_id']
        participants = project['participants']
        
        if project_id not in projects_dict:
            projects_dict[project_id] = {
                "name": project_name,
                "gepris_id": project_id,
                "participants": set(participants)
            }
        else:
            projects_dict[project_id]["participants"].update(participants)
    
    # Konvertiere das Dictionary in eine Liste von Projekten
    projects = []
    for proj_info in projects_dict.values():
        # Erstelle die Teilnehmer-ID-Liste basierend auf human_info_dict
        participants_ids = [human_info_dict.get(p, {}).get('qid', '') for p in proj_info["participants"]]
        
        projects.append({
            "name": proj_info["name"],
            "gepris_id": proj_info["gepris_id"],
            "participants_id": participants_ids
        })
    
    return projects

# Hauptfunktion
def main(input_file, output_file):
    data = load_json(input_file)
    structured_data = restructure_json(data)
    save_json(structured_data, output_file)
    print(f'Daten wurden erfolgreich bearbeitet und in "{output_file}" gespeichert.')

if __name__ == "__main__":
    main(input_file, output_file)


Daten wurden erfolgreich bearbeitet und in "GeprisJsonAndCsv/output_restructured_projects_with_ids.json" gespeichert.


json mit nur Projekten, die angelegt werden müssen

In [46]:
import json

# Datei- und Liste-Definitionen
input_file = 'GeprisJsonAndCsv/output_restructured_projects.json'
output_file = 'GeprisJsonAndCsv/output_restructured_projects_filtered.json'

# Liste der gepris_ids, die beibehalten werden sollen
non_existing_entries = [
    '433110396', '492813820', '509313233', '521476232', '520751609', '5310710',
    '198305071', '218318381', '314772579', '431352836', '230488449', '505997786',
    '224619622', '326552732', '290363600', '24122740', '5413368', '453182863',
    '453229399', '513892404', '242504939', '316102599', '418004173', '445552570',
    '516780480', '418598556', '418603037', '424650015', '424795268', '447603908',
    '456668568', '458957343', '319835486', '325443116', '221270173', '183605059',
    '284237345', '255821879', '507302435', '270041755', '491261247'
]

# Dictionary mit weiteren Informationen zu den Forschern
human_info_dict = {
    'Konrad Förstner': {'description': 'bioinformatician', 'qid': 'Q18744528'},
    'Anke Becker': {'description': 'German university teacher', 'qid': 'Q21253882'},
    'Peer Bork': {'description': 'German biologist and bioinformatician', 'qid': 'Q7160367'},
    'Thomas Clavel': {'description': 'researcher', 'qid': 'Q40442760'},
    'Alexander Goesmann': {'description': 'researcher', 'qid': 'Q52422599'}
}

# Lade die JSON-Daten aus einer Datei
def load_json(filename):
    with open(filename, 'r', encoding='utf-8') as file:
        return json.load(file)

# Speichere die bearbeiteten JSON-Daten in eine Datei
def save_json(data, filename):
    with open(filename, 'w', encoding='utf-8') as file:
        json.dump(data, file, ensure_ascii=False, indent=2)

# Funktion zur Umstrukturierung der JSON-Daten
def filter_projects(data):
    filtered_projects = []

    for project in data:
        project_id = project.get('gepris_id')
        if project_id in non_existing_entries:
            # Erstelle die Teilnehmer-ID-Liste basierend auf human_info_dict
            participants_ids = [
                human_info_dict.get(p, {}).get('qid', '')
                for p in project.get('participants', [])
                if human_info_dict.get(p) is not None
            ]
            
            filtered_projects.append({
                "name": project.get('name'),
                "gepris_id": project_id,
                "participants_id": participants_ids
            })

    return filtered_projects

# Hauptfunktion
def main(input_file, output_file):
    data = load_json(input_file)
    filtered_data = filter_projects(data)
    save_json(filtered_data, output_file)
    print(f'Daten wurden erfolgreich gefiltert und in "{output_file}" gespeichert.')

if __name__ == "__main__":
    main(input_file, output_file)


Daten wurden erfolgreich gefiltert und in "GeprisJsonAndCsv/output_restructured_projects_filtered.json" gespeichert.


In [49]:
from wikidataintegrator import wdi_core  # wdi_edit, wdi_login,
import json
import logging

# Konfiguration des Loggings
logging.basicConfig(level=logging.DEBUG)

# Deine Wikidata-Anmeldedaten
WDUSER = 'Lennart237THKoeln@AWDE-Bot'
WDPASS = 's1vrjdampl8djincgnk5ic16s7gf8khq'

# Anmelden
logging.info("Anmeldung bei Wikidata...")
try:
    login_instance = wdi_login.WDLogin(user=WDUSER, pwd=WDPASS)
    logging.info("Erfolgreich bei Wikidata angemeldet.")
except Exception as e:
    logging.error(f"Fehler bei der Anmeldung: {e}")
    exit()

# Lese die JSON-Datei
with open('GeprisJsonAndCsv/output_restructured_projects_with_ids.json', 'r') as file:
    projects = json.load(file)

# Erstelle eine Liste von Items, die hochgeladen werden sollen
items_to_upload = []

for project in projects:
    # Erstelle ein neues Wikidata-Item
    item_data = {
        'labels': {
            'de': project["name"],
        },
        'claims': {
            'P31': ['Q1298668']  # Setze den Typ des Projekts
        }
    }

    # Füge die GEPRIS ID als Statement hinzu (hier als Beispiel, dies hängt von den tatsächlichen Anforderungen ab)
    item_data['claims']['P4870'] = [project["gepris_id"]]

    # Füge alle Teilnehmer IDs als Statements hinzu
    for participant_id in project["participants_id"]:
        item_data.setdefault('claims', {}).setdefault('P710', []).append(participant_id)

    items_to_upload.append(item_data)

print(items_to_upload)

# # Hochladen der Items
# for item_data in items_to_upload:
#     try:
#         item = wdi_core.WDItemEngine(data=item_data, login=login_instance)
#         item.write()
#         logging.info(f"Projekt '{item_data['labels']['de']}' wurde erfolgreich erstellt.")
#     except Exception as e:
#         logging.error(f"Fehler beim Erstellen des Projekts '{item_data['labels']['de']}': {e}")

# logging.info("Alle Projekte wurden erfolgreich erstellt.")


INFO:root:Anmeldung bei Wikidata...
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.wikidata.org:443
DEBUG:urllib3.connectionpool:https://www.wikidata.org:443 "GET /w/api.php?action=query&meta=tokens&type=login&format=json HTTP/1.1" 200 110
DEBUG:urllib3.connectionpool:https://www.wikidata.org:443 "POST /w/api.php HTTP/1.1" 200 95
DEBUG:urllib3.connectionpool:https://www.wikidata.org:443 "GET /w/api.php?action=query&meta=tokens&format=json HTTP/1.1" 200 109
INFO:root:Erfolgreich bei Wikidata angemeldet.


[{'labels': {'de': 'sRNARegNet - Vergleichende Analyse der regulatorischen Netzwerke kleiner RNA in Gammaproteobacteria'}, 'claims': {'P31': ['Q1298668'], 'P123': ['433110396'], 'P50': ['Q18744528']}}, {'labels': {'de': 'PIXLS - Preprint Information eXtraction for Life Sciences'}, 'claims': {'P31': ['Q1298668'], 'P123': ['492813820'], 'P50': ['Q18744528']}}, {'labels': {'de': 'Automatic Quality Assessment: NLP-Verfahren zur semantischen Kartierung von Lebenswissenschaftlichen Texten (AQUAS)'}, 'claims': {'P31': ['Q1298668'], 'P123': ['509313233'], 'P50': ['Q18744528']}}, {'labels': {'de': 'NFDI4Microbiota - Nationale Forschungsdateninfrastruktur für Mikrobiota-Forschung'}, 'claims': {'P31': ['Q1298668'], 'P123': ['460129525'], 'P50': ['Q52422599', 'Q18744528', 'Q7160367', 'Q40442760', 'Q21253882']}}, {'labels': {'de': 'Base4NFDI - Basisdienste für die NFDI (NFDI4Microbiota)'}, 'claims': {'P31': ['Q1298668'], 'P123': ['521476232'], 'P50': ['Q18744528']}}, {'labels': {'de': 'SP4 - Wechse